# DeepDream
DeepDream is an artistic image-modification technique that uses the representations learned by convolutional neural networks. It was first released by Google in the summer of 2015, as an implementation written using the Caffe deep-learning library (this
was several months before the first public release of TensorFlow).4 It quickly became an internet sensation thanks to the trippy pictures it could generate, full of algorithmic pareidolia artifacts, bird feathers, and dog eyes—a byproduct of the fact that the DeepDream convnet was trained on ImageNet, where dog breeds and bird species are vastly overrepresented.

The DeepDream algorithm is almost identical to the convnet filter-visualization technique, consisting of running a convnet in reverse: doing gradient ascent on the input to the convnet in order to maximize the activation of a
specific filter in an upper layer of the convnet. DeepDream uses this same idea, with a few simple differences:
 - With DeepDream, we try to maximize the activation of entire layers rather than that of a specific filter, thus mixing together visualizations of large numbers of features at once.
 - We start not from blank, slightly noisy input, but rather from an existing image—thus the resulting effects latch on to preexisting visual patterns, distorting elements of the image in a somewhat artistic fashion.
 - The input images are processed at different scales (called *octaves*), which improves the quality of the visualizations.
 

### Implement DeepDream in Keras
We’ll start from a convnet pretrained on ImageNet. In Keras, many such convnets are available: `VGG16`, `VGG19`, `Xception`, `ResNet50`, and so on. You can implement Deep- Dream with any of them, but your convnet of choice will naturally affect your visualizations, because different convnet architectures result in different learned features. The convnet used in the original DeepDream release was an Inception model, and in practice Inception is known to produce nice-looking DeepDreams, so we’ll use the Inception V3 model that comes with Keras.

### Load The Pretrained Inception V3 Model

In [4]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)  #We won’t be training the model, so this command disables all trainingspecific operations.

model= inception_v3.InceptionV3(weights='imagenet', include_top=False) #Builds the Inception V3 network, without its 
                                                                       #convolutional base. The model will be loaded with
                                                                       #pretrained ImageNet weights.


87916544/87910968 [==============================] - 79s 1us/step


Next, we’ll compute the loss: the quantity we’ll seek to maximize during the gradient-ascent process. For filter visualization, we tried to maximize the value of a specific filter in a specific layer. Here, we’ll simultaneously maximize the activation of all filters in a number of layers. Specifically, we’ll maximize a weighted sum of the L2 norm of the activations of a set of high-level layers. The exact set of layers you choose (as well as their contribution to the final loss) has a major influence on the visuals we’ll be able to produce, so we want to make these parameters easily configurable. Lower layers result in geometric patterns, whereas higher layers result in visuals in which we can recognize some classes from ImageNet (for example, birds or
dogs). We’ll start from a somewhat arbitrary configuration involving four layers—but we’ll definitely want to explore many different configurations later.

### Set Up The DeepDream Configuration

In [9]:
layer_contributions={          #Dictionary mapping layer names to a coefficient quantifying how much the layer’s 
    'mixed2': 0.2,            #activation contributes to the loss you’ll seek to maximize. Note that the layer names are
    'mixed3': 3.0,            #hardcoded in the built-in Inception V3 application. We can list all layer names using 
    'mixed4': 2.0,            #model.summary().
    'mixed5': 1.5
}

Now, let’s define a tensor that contains the loss: the weighted sum of the L2 norm of the activations of the layers in the above listing.

### Define The Loss to be Maximized

In [10]:
layers_dict= dict([(layer.name, layer) for layer in model.layers]) #Creates a dictionary that maps layer names to layer instances
layers_dict

{'input_1': <keras.engine.input_layer.InputLayer at 0x1f76fd74898>,
 'conv2d_1': <keras.layers.convolutional.Conv2D at 0x1f76fd74a58>,
 'batch_normalization_1': <keras.layers.normalization.BatchNormalization at 0x1f76fd74a20>,
 'activation_1': <keras.layers.core.Activation at 0x1f76fd74e10>,
 'conv2d_2': <keras.layers.convolutional.Conv2D at 0x1f76fd9b550>,
 'batch_normalization_2': <keras.layers.normalization.BatchNormalization at 0x1f76fe0d8d0>,
 'activation_2': <keras.layers.core.Activation at 0x1f76fe0d668>,
 'conv2d_3': <keras.layers.convolutional.Conv2D at 0x1f707555518>,
 'batch_normalization_3': <keras.layers.normalization.BatchNormalization at 0x1f707593f60>,
 'activation_3': <keras.layers.core.Activation at 0x1f7075ba7b8>,
 'max_pooling2d_1': <keras.layers.pooling.MaxPooling2D at 0x1f7078e5470>,
 'conv2d_4': <keras.layers.convolutional.Conv2D at 0x1f7078b2e80>,
 'batch_normalization_4': <keras.layers.normalization.BatchNormalization at 0x1f7084fd828>,
 'activation_4': <keras.

In [12]:
loss=K.variable(0.) #We'll define the loss by adding layer contributions to this scalar variable
for layer_name in layer_contributions:
    coeff= layer_contributions[layer_name]
    activation= layers_dict[layer_name].output #Retrieves the layer’s output

    scaling= K.prod(K.cast(K.shape(activation), 'float32'))
    loss += coeff*K.sum(K.square(activation[:, 2: -2, 2: -2, :]))/scaling  #Adds the L2 norm of the features of a layer
                                        #to the loss. We avoid border artifacts by only involving nonborder pixels in the loss

Next, we can set up the gradient-ascent process.

### Gradient-Ascent Process

In [13]:
dream= model.input #This tensor holds the generated image: the dream

grads= K.gradients(loss, dream)[0]   #Computes the gradients of the dream with regard to the loss

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7) #Normalizes the gradients (important trick)

outputs= [loss, grads]                                 #Sets up a Keras function to retrieve the value of
fetch_loss_and_grads= K.function([dream], outputs)     #the loss and gradients, given an input image

def eval_loss_and_grads(x):
    outs= fetch_loss_and_grads([x])
    loss_value= outs[0]
    grad_value= outs[1]
    return loss_value, grad_value

def gradient_ascent(x, iterations, step, max_loss= None):
    for i in range(iterations):
        loss_value, grads_value= eval_loss_and_graphs(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('... Loss Value at',i,': ', loss_value)
        x+= step+grad_values
    return x

Finally: the actual DeepDream algorithm. First, we define a list of scales (also called octaves) at which to process the images. Each successive scale is larger than the previous one by a factor of 1.4 (it’s 40% larger): we start by processing a small image and then increasingly scale it up.
![image](https://user-images.githubusercontent.com/13174586/51905645-c28dc800-23e7-11e9-89e7-30ffd9fc6e47.png)
